In [1]:
#importing modules
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
#importing under sampling dataset
ecommerce_df = pd.read_csv(Path('Resources/Cleaned_data/under_sampling_data.csv'))

In [13]:
#looking the few rows
ecommerce_df.head()

,Administrative_Avg,Informational_Avg,ProductRelated_Avg,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,44.541667,40.666667,53.708333,0.000000,0.018182,0.000000,0.0,Dec,2,2,2,1,Returning_Visitor,False,False
1,0.000000,0.000000,53.166667,0.000000,0.050000,0.000000,0.0,Mar,1,1,6,1,Returning_Visitor,True,False
2,0.000000,0.000000,220.678571,0.014286,0.033333,0.000000,0.0,Dec,1,1,3,1,Returning_Visitor,False,False
3,5.666667,0.000000,24.086352,0.009524,0.040317,0.000000,0.0,Dec,2,2,1,2,Returning_Visitor,False,False
4,22.260000,0.000000,30.232723,0.000000,0.008337,17.634346,0.0,Nov,2,2,9,2,Returning_Visitor,False,False


In [14]:
ecommerce_df["Revenue"].value_counts()

False    1908
True     1908
Name: Revenue, dtype: int64

In [15]:
#shape of dataframe
ecommerce_df.shape

(3816, 15)

In [16]:
# create feature matrix (X)
#selecting all columns except 'Revenue'
X=ecommerce_df.drop('Revenue', axis=1) 

# create response vector (y)
##selecting 'Revenue' column
y=ecommerce_df['Revenue'].values 

In [17]:
# Seperating categorical columns
cat_columns=['Month','OperatingSystems', 'Browser', 'Region',
             'TrafficType','VisitorType','Weekend']

In [18]:
# One-hot encoding the entire dataframe
X_dummies = pd.get_dummies(X, columns = cat_columns)
print(X_dummies.columns)
X_dummies.head()

Index(['Administrative_Avg', 'Informational_Avg', 'ProductRelated_Avg',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month_Aug',
       'Month_Dec', 'Month_Feb', 'Month_Jul', 'Month_June', 'Month_Mar',
       'Month_May', 'Month_Nov', 'Month_Oct', 'Month_Sep',
       'OperatingSystems_1', 'OperatingSystems_2', 'OperatingSystems_3',
       'OperatingSystems_Other', 'Browser_1', 'Browser_10', 'Browser_2',
       'Browser_4', 'Browser_5', 'Browser_6', 'Browser_Other', 'Region_1',
       'Region_2', 'Region_3', 'Region_4', 'Region_5', 'Region_6', 'Region_7',
       'Region_8', 'Region_9', 'TrafficType_1', 'TrafficType_10',
       'TrafficType_11', 'TrafficType_13', 'TrafficType_2', 'TrafficType_3',
       'TrafficType_4', 'TrafficType_5', 'TrafficType_6', 'TrafficType_8',
       'TrafficType_Other', 'VisitorType_New_Visitor', 'VisitorType_Other',
       'VisitorType_Returning_Visitor', 'Weekend_False', 'Weekend_True'],
      dtype='object')


,Administrative_Avg,Informational_Avg,ProductRelated_Avg,BounceRates,ExitRates,PageValues,SpecialDay,Month_Aug,Month_Dec,Month_Feb,...,TrafficType_4,TrafficType_5,TrafficType_6,TrafficType_8,TrafficType_Other,VisitorType_New_Visitor,VisitorType_Other,VisitorType_Returning_Visitor,Weekend_False,Weekend_True
0,44.541667,40.666667,53.708333,0.000000,0.018182,0.000000,0.0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
1,0.000000,0.000000,53.166667,0.000000,0.050000,0.000000,0.0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
2,0.000000,0.000000,220.678571,0.014286,0.033333,0.000000,0.0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
3,5.666667,0.000000,24.086352,0.009524,0.040317,0.000000,0.0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
4,22.260000,0.000000,30.232723,0.000000,0.008337,17.634346,0.0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
